In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('arrests_national_juvenile.csv')
data = df[df['year'] == 2016][['offense_name', 'total_male', 'total_female']]
data

,offense_name,total_male,total_female
0,Arson,1760,328
1,Aggravated Assault,16997,5918
2,Simple Assault,66360,38712
3,Burglary,23307,3071
4,Curfew and Loitering Law Violations,19218,8319
5,Disorderly Conduct,34438,19449
6,Drug Abuse Violations,61781,18503
7,Drunkenness,2746,1120
8,Drive Under the Influence,3913,1345
9,Embezzlement,343,207


In [3]:
def chi_square(data):
    result = []
    statistics = 0
    columns = data.columns.tolist()
    b_0 = data[columns[1]].sum()
    b_1 = data[columns[2]].sum()
    total = b_0 + b_1
    
    for _, row in data.iterrows():
        total_row = row[columns[1]] + row[columns[2]]
        a = (total_row * b_0) / total
        b = (total_row * b_1) / total
        result.append([a, b])
        statistics += ((row[columns[1]] - a) ** 2) / a
        statistics += ((row[columns[2]] - b) ** 2) / b
        
    return statistics, pd.DataFrame(result)
    
def Pearson(x, n):
    return (x / (x + n)) ** 0.5

def Cramer(x, n, m, k):
    return (x / (n * min((m - 1), (k - 1)))) ** 0.5

In [4]:
chi = chi_square(data)

chi[0]

38672.962759139686

- Для степени свободы X0.95;29 критическая область имеет вид: (42.6; ∞) - значение статистики попадает в критическую область, гипотеза о независимости признаков отвергается на уровне значимости 0.05

In [5]:
n = sum(data.sum().values.tolist()[1:])
p = Pearson(chi[0], n)
c = Cramer(chi[0], n, 2, 30)

n, p, c

(742483, 0.22250247507871126, 0.2282235545906968)

- Значения коэффициентов Пирсона и Крамера близки к нулю, что говорит о достаточно слабой силе выявленной связи.

In [6]:
# Расчет статистики для первых 2 категорий преступлений

chi = chi_square(data[:2])
n = sum(data[:2].sum().values.tolist()[1:])
p = Pearson(chi[0], n)
c = Cramer(chi[0], n, 2, 2)
chi[0], n, p, c

(104.5168431131478, 25003, 0.0645195397119742, 0.06465425047027804)

- Для степени свободы X0.95;1 критическая область имеет вид: (3.84; ∞) - значение статистики попадает в критическую область, гипотеза о независимости признаков отвергается на уровне значимости 0.05
- Значения коэффициентов Пирсона и Крамера близки к нулю, что говорит о достаточно слабой силе выявленной связи.

In [7]:
# Расчет статистики для первых 15 категорий преступлений

chi = chi_square(data[:15])
n = sum(data[:15].sum().values.tolist()[1:])
p = Pearson(chi[0], n)
c = Cramer(chi[0], n, 2, 15)
chi[0], n, p, c

(10190.716521622586, 336432, 0.1714643548148431, 0.17404185830036273)

- Для степени свободы X0.95;14 критическая область имеет вид: (23,7; ∞) - значение статистики попадает в критическую область, гипотеза о независимости признаков отвергается на уровне значимости 0.05
- Значения коэффициентов Пирсона и Крамера близки к нулю, что говорит о достаточно слабой силе выявленной связи.

In [8]:
def goodman_kraskel(data=data):
    total_male = sum(data['total_male']) 
    total_female = sum(data['total_female'])
    data['total'] = data['total_male'] + data['total_female']
    total =  total_male + total_female
    sum_nj_b = total_male ** 2 / total + total_female ** 2 / total
    p1_b = 1 - (sum_nj_b / total)
    sum_nij_b = 0
    for _, row in data.iterrows():
        ni = row['total_male'] + row['total_female']
        sum_nij_b += row['total_male'] ** 2 / ni 
        sum_nij_b += row['total_female'] ** 2 / ni
    p2_b = 1 - (sum_nij_b / total)
    p1_b = round(p1_b, 3)
    p2_b = round(p2_b, 3)
    tb = (p1_b - p2_b) / p1_b
    print(f'p1_b = {round(p1_b, 3)}, p2_b = {round(p2_b, 3)}, tb = {round(tb, 3)}')
    
    p1_a = 1 - sum([(i ** 2) / total for i in data['total']]) / total
    sum_nij_a = sum([j ** 2 / total_male for j in data['total_male']])
    sum_nij_a += sum([j ** 2 / total_female for j in data['total_female']])
    p2_a = 1 - (sum_nij_a / total)
    p1_a = round(p1_a, 3)
    p2_a = round(p2_a, 3)
    ta = (p1_a - p2_a) / p1_a
    print(f'p1_a = {p1_a}, p2_a = {p2_a}, ta = {round(ta, 3)}')

In [9]:
goodman_kraskel()

p1_b = 0.426, p2_b = 0.404, tb = 0.052
p1_a = 0.899, p2_a = 0.896, ta = 0.003


- Построенные оценки позволяют сказать, что прогноз модальной категории признака B (категория преступления) улучшится на 5,2%, если при прогнозировании будет учтено значение признака A (пол преступника), а прогноз модальной категории признака A улучшится на 0,3%, если при прогнозировании будет учтено значение признака B.
- Таким образом, значение статистики Хи-квадрат (X2 = 38672.96, критическая область: (χ2_0,95(29); +∞) = (42,6; +∞)) указывает на наличие зависимости между категорией совершаемого преступления и полом преступника. Значения коэффициентов взаимной сопряженности Пирсона (P = 0.22) и Крамера (C = 0.23) указывают на слабую силу связи между признаками. Значения мер прогноза Гудмана и Гудмана-Краскела указывают, что прогноз категории преступления улучшится при учете пола преступника ((λ_B ) ̂=0.008, τ ̂_B= 0.052), а прогноз категории пола преступника улучшится при учете категории преступления ((λ_A ) ̂=0.021, τ ̂_A= 0.003).